### Ingestión del archivo "language_rol.json"

In [0]:
dbutils.widgets.text("p_environment","")
v_environment = dbutils.widgets.get("p_environment")

In [0]:
dbutils.widgets.text("p_file_date","2024-12-16")
v_file_date = dbutils.widgets.get("p_file_date")

In [0]:
%run "../includes/configuration"

In [0]:
%run "../includes/common_functions"

#### Paso 1 - Leer archvio JSON usando "DataFrameReader" de Spark

In [0]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType

In [0]:
languages_roles_schema = StructType(fields= [
    StructField("roleId", IntegerType(), True),
    StructField("languageRole", StringType(), True)
])                       

In [0]:
languages_roles_df = spark.read \
        .schema(languages_roles_schema) \
        .option("multiLine", True) \
        .json(f"{bronze_folder_path}/{v_file_date}/language_role.json") 

#### Pase 2 - Renombrar las columnas y añadir nuevas columnas
1. "roleId" renombrar a "role_id"
2. "languageRole" renombrar a "language_role"
33. Agregar la columnas "ingestion_date" y "environment"

In [0]:
from pyspark.sql.functions import current_timestamp, lit

In [0]:
languages_roles_with_columns_df = add_ingestion_date(languages_roles_df) \
    .withColumnsRenamed({"roleId" : "role_id",
                         "languageRole": "language_role"}) \
    .withColumn("environment", lit(v_environment)) \
    .withColumn("file_date", lit(v_file_date))


#### Paso 3 - Escribir datos en el DataLake en formato "Parquet"

In [0]:
languages_roles_with_columns_df.write.mode("overwrite").format("delta").saveAsTable("movie_silver.languages_roles")
# languages_roles_with_columns_df.write.mode("overwrite").format("parquet").saveAsTable("movie_silver.languages_roles")
# languages_roles_with_columns_df.write.mode("overwrite").parquet(f"{silver_folder_path}/languages_roles")

In [0]:
%sql
SELECT * FROM movie_silver.languages_roles;

In [0]:
dbutils.notebook.exit("Success")